In [10]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as e:
    credential = InteractiveBrowserCredential()

In [11]:
from azure.ai.ml import MLClient

ml_client = MLClient.from_config(credential=credential)

Found the config file in: .\config.json


Deploy to online endpoint

In [12]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M")

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for model deployment",
    auth_mode="key"
)

In [13]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-01051009.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-01051009.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-01051009', 'description': 'Online endpoint for model deployment', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/6e5b4e5f-b5c5-4461-ae17-7ae3bc1a747e/resourcegroups/rg-app-with-model-859731b6b1ac4414b1/providers/microsoft.machinelearningservices/workspaces/mlw-859731b6b1ac4414b1/onlineendpoints/endpoint-01051009', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6e5b4e5f-b5c5-4461-ae17-7ae3bc1a747e/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:94a49fe4-c295-4222-89b7-f0b90eafa87a:e55c289c-cf25-40c6-9d6b-6240d315711a?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/6e5b4e5f-b5c5-4461-ae17-7ae3bc1a747e/

In [14]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

model = Model(
    path="./mlflow-model",
    type=AssetTypes.MLFLOW_MODEL,
    description="diabetes mlflow model"
)

In [15]:
deployment_name = "diabetes-model-deployment"

deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS1_v2",
    instance_count=1
)

In [16]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint endpoint-01051009 exists


.............................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpoint-01051009', 'type': 'Managed', 'name': 'diabetes-model-deployment', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6e5b4e5f-b5c5-4461-ae17-7ae3bc1a747e/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/od:94a49fe4-c295-4222-89b7-f0b90eafa87a:09a54f55-db21-4ab2-ae18-cb364a28feb4?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/6e5b4e5f-b5c5-4461-ae17-7ae3bc1a747e/resourceGroups/rg-app-with-model-859731b6b1ac4414b1/providers/Microsoft.MachineLearningServices/workspaces/mlw-859731b6b1ac4414b1/onlineEndpoints/endpoint-01051009/deployments/diabetes-model-deployment', 'Resource__source_path': None, 'base_path': 'c:\\Users\\ciroalfonsom\\learning\\diabetes-prediction-app\\model-creation', 'creation_context': None, 'serialize': <msrest.serializ

In [17]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file="sample-data.json"
)

print(response)

[1]
